In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10795,2024-04-19,Eua Nba,20:00,Miami Heat,Chicago Bulls,1.81,2.06,205.5,1.93,1.93,-2.5,2.00,2.35,QwG418yo,0.552486,0.485437,0.518135,0.518135,0.037923,0.2,0.8,NaN,NaN,228.902,19.542028,0.085373,222.444,39.026971,0.175446,219.388,327.700,235.52,229.62,0.0,0.0,0.0,0.0,0.091357,0.000000,0.113787,-2.14,-0.428,-1.892523,0.522214,0.3,-0.222214,2.58,0.516,2.054264,0.427962,0.5,0.072038
10796,2024-04-19,Eua Nba,22:30,New Orleans Pelicans,Sacramento Kings,2.02,1.85,210.5,1.80,1.95,1.5,1.89,1.63,nktbZsdD,0.495050,0.540541,0.555556,0.512821,0.035590,0.4,0.6,NaN,NaN,233.836,27.703742,0.118475,247.864,17.843134,0.071988,224.722,263.358,204.58,247.80,0.0,0.0,0.0,0.0,0.062123,0.056569,0.104459,-2.92,-0.584,-1.746575,0.590956,0.4,-0.190956,2.21,0.442,1.923077,0.425080,0.4,-0.025080
10797,2024-04-19,Europa Euroliga,15:30,Baskonia,Virtus Bologna,1.77,2.10,168.5,1.91,1.91,-2.5,2.00,2.40,Ofgu9uq1,0.564972,0.476190,0.523560,0.523560,0.041162,0.6,0.4,NaN,NaN,221.694,81.550752,0.367853,191.816,71.425099,0.372363,127.024,216.406,126.92,312.89,0.0,0.0,0.0,0.0,0.120592,0.000000,0.128565,-0.45,-0.090,-8.555556,0.613413,0.7,0.086587,-0.33,-0.066,-16.666667,0.426707,0.3,-0.126707
10798,2024-04-19,Austrália Nbl1 Leste,07:00,Sydney Comets,Inner West Bulls,1.73,2.01,175.5,1.84,1.86,-1.5,1.85,2.12,fo84MofJ,0.578035,0.497512,0.543478,0.537634,0.075547,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,167.32,0.00,0.0,0.0,0.0,0.0,0.105877,0.007644,0.096181,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10799,2024-04-19,Austrália Nbl1 Oeste,09:30,Mandurah Magic,Beeliar Boodjar,1.58,2.29,188.5,1.85,1.85,-1.5,1.86,2.10,IHf6TvDo,0.632911,0.436681,0.540541,0.540541,0.069593,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,252.35,0.00,0.0,0.0,0.0,0.0,0.259455,0.000000,0.085710,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10800,2024-04-19,Bulgária Nbl,13:00,Academic Plovdiv,Minyor,1.75,1.98,162.5,1.84,1.86,-2.5,2.02,2.38,WWnYEB7a,0.571429,0.505051,0.543478,0.537634,0.076479,0.6,0.4,NaN,NaN,329.406,233.774709,0.709686,173.912,51.105563,0.293859,158.784,254.334,207.36,237.36,0.0,0.0,0.0,0.0,0.087204,0.007644,0.115708,1.83,0.366,2.049180,0.000000,0.0,0.000000,-0.25,-0.050,-19.600000,0.000000,0.0,0.000000
10801,2024-04-19,Coreia Do Sul Kbl,07:00,KCC Egis,Wonju DB,1.70,2.13,172.5,1.88,1.90,-3.5,2.05,2.49,nRgChH6U,0.588235,0.469484,0.531915,0.526316,0.057719,1.0,0.6,NaN,NaN,165.790,31.504495,0.190027,142.698,30.303574,0.212362,183.142,143.962,129.01,189.00,0.0,0.0,0.0,0.0,0.158776,0.007483,0.137060,1.46,0.292,2.397260,0.627608,0.9,0.272392,-0.03,-0.006,-188.333333,0.687082,0.6,-0.087082
10802,2024-04-19,Filipinas Copa Das Filipinas,08:30,Ginebra Kings,TNT Tropang Giga,1.70,2.08,184.5,1.82,1.88,-2.5,1.93,2.28,nHCoMfm4,0.588235,0.480769,0.549451,0.531915,0.069005,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,201.48,164.68,0.0,0.0,0.0,0.0,0.142170,0.022933,0.117571,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10803,2024-04-19,França Pro B,15:00,Aix-Maurienne,ASA,1.71,2.03,165.5,1.85,1.85,-2.5,1.97,2.45,SnOarLvS,0.584795,0.492611,0.540541,0.540541,0.077406,0.4,0.0,NaN,NaN,238.460,73.460084,0.308060,161.628,39.283135,0.243047,180.686,178.376,220.15,217.60,0.0,0.0,0.0,0.0,0.121002,0.000000,0.153580,1.75,0.350,2.028571,0.000000,0.0,0.000000,2.03,0.406,2.536946,0.000000,0.0,0.000000
10804,2024-04-19,França Pro B,15:3

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Miami Heat,Chicago Bulls,205.5,1.93,1.0000,Over
1,15:30,Europa Euroliga,Baskonia,Virtus Bologna,168.5,1.91,1.0000,Over
2,07:00,Austrália Nbl1 Leste,Sydney Comets,Inner West Bulls,175.5,1.84,0.9973,Over
3,09:30,Austrália Nbl1 Oeste,Mandurah Magic,Beeliar Boodjar,188.5,1.85,0.9984,Over
4,13:00,Bulgária Nbl,Academic Plovdiv,Minyor,162.5,1.84,1.0000,Over
5,07:00,Coreia Do Sul Kbl,KCC Egis,Wonju DB,172.5,1.88,1.0000,Over
6,08:30,Filipinas Copa Das Filipinas,Ginebra Kings,TNT Tropang Giga,184.5,1.82,1.0000,Over
7,15:00,França Pro B,Aix-Maurienne,ASA,165.5,1.85,1.0000,Over
8,15:30,França Pro B,St. Chamond,Chalons-Reims,163.5,1.84,0.9999,Over
9,15:00,França Lfb Feminina,Landerneau Bretagne F,Flammes Carolo F,145.5,1.85,1.0000,Over
